1. Train KNN model using Cosine similarity with user rating data

We want to choose the similar user to every user.
Find the similar user by all user ratings.(calculate by cosine similarity of person A's rating to person B's ratings)
If the ratings are similar, these two people have similar tatse.

In [1]:
import pandas as pd
columns = ["userId","movieId","rating"]
df = pd.read_csv("kafka-consumer/data/user_ratings_ongoing.csv",names=columns) #with 370k data
df = df[1:]
df.head()


/tmp/ipykernel_99470/2485621834.py:3: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("kafka-consumer/data/user_ratings_ongoing.csv",names=columns) #with 370k data


,userId,movieId,rating
1,468282,jurassic+park+1993,4
2,263188,pulp+fiction+1994,3
3,296237,life+is+rosy+1987,5
4,255439,speed+1994,4
5,283675,the+shawshank+redemption+1994,4


In [2]:
df.shape
df = df[:10000]

In [3]:
import pandas as pd
from surprise import Reader
from surprise import KNNWithMeans

In [4]:
from surprise import Dataset

# Loads rating data for training 
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader)


In [5]:
# To use user-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": True,
}

# Training algorithm using KNNwithMeans, to find the closest users to user A
algo = KNNWithMeans(sim_options=sim_options) #(k=40, min_k=1, sim_options={}, verbose=True, **kwargs)

In [6]:
#split training, validation dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
trainset, validationset = train_test_split(data, test_size=.20)
trainset

In [7]:
algo.fit(trainset)
#predictions = algo.test(validationset)
#accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [8]:
#formatting result
def formatting_result(predictions): 
    columns = ["userId", "movieId","rating","est","details"]
    df_result = pd.DataFrame(predictions,columns = columns)
    df_result = df_result[["userId", "movieId","rating","est"]]
    # add rank
    # each user, we rank the rightest ratings for website recommendations
    df_result["rank"] = df_result.groupby("userId")["est"].rank("dense", ascending=False)
    return df_result

2. Get movie list

In [9]:
def getMovieFile():
    # read all the movie ids
    import pandas as pd
    df_movies = pd.read_csv("kafka-consumer/data/clean_data_1.csv")
    df_movies.shape

    # df_movies.head()
    ser_movies = df_movies["id"] 
    df_movies.head(5)
    return ser_movies
ser_movies = getMovieFile()

3. Recommendation

In [10]:
#For the recmmoendation for a user, create userTestSet for prediction
    # list of tuples
    #[('469629', 'casino+1995', 5.0),
    # ('871227', 'bedknobs+and+broomsticks+1971', 3.0)]

def makeSet(userid): # input should be a number instead of str
    userid = str(userid)
    userTestSet = list()
    for movie in ser_movies:
        tuple = (userid, movie, 0)
        userTestSet.append(tuple)
    return userTestSet

In [11]:

#choose top 20 ratings of movies for one user
def chooseTop20(userid,df_result):
    is_user = df_result["userId"]== str(userid)
    df_user = df_result[is_user]
    df_user = df_user.sort_values(["rank"], ascending=True)
    # print(df_user)
    if (df_user.shape[0]>20):
        df_user  = df_user[:20]
    return df_user

# For cold start users/ users who didnt occur in the rating file
# give them the most popular movies 
def Top20inAllMovie(df):
    # the movie occured the most in the movie rating file
    df_top20inAll = df["movieId"].value_counts()[:20]
    return df_top20inAll

In [12]:
def predictAndRecommend(userId):
    userId = str(userId)
    # check if the user didn't rate before, return top most popular movie directly
    if userId not in df["userId"].values:
        return Top20inAllMovie(df)
    # if the user has rated before, predict her Top 20 rated movies 
    userTestSet = makeSet(userId)
    predictions2 = algo.test(userTestSet)
    df_result = formatting_result(predictions2)
    df_user = chooseTop20(userId,df_result)
    df.to_csv('model1_data/out.csv')
    user_movieList = df_user['movieId'].tolist()
    return user_movieList


In [13]:
#test with user 
predictAndRecommend(468282)


['up+2009',
 'sleepless+in+seattle+1993',
 'about+schmidt+2002',
 'lock_+stock+and+two+smoking+barrels+1998',
 'the+sound+of+music+1965',
 'a+christmas+story+1983',
 'the+avengers+2012',
 'the+lion+king+1994',
 'forrest+gump+1994',
 'the+child+2005',
 'copycat+1995',
 'the+great+dictator+1940',
 'the+rock+1996',
 'austin+powers+the+spy+who+shagged+me+1999',
 'some+folks+call+it+a+sling+blade+1994',
 'empire+of+the+sun+1987',
 'the+usual+suspects+1995',
 'sleepy+hollow+1999',
 'babe+1995',
 'beauty+and+the+beast+1991']

# TODO

In [14]:


#TODO: prediction accuracy : split the set. partly is wachted, partly is prepared to watch
        # compare with prepare to wach with my recommendation 
#TODO5 : havent filter out the movie the user rated before


#WAIT : update movie data clean data
#WAIT : update more rating data

#finished : cold start user give them the most popular movies 
#fihished : what if the user id is not in the rating dataset

#def find_most_poplular20():
    # the movie occured the most in the movie rating file

# for how long would the rating data covers all user

User size:

 currently we have 66k users.

 total is about 1 million customers 

#   Tuning the Algorithm Parameters
https://realpython.com/build-recommendation-engine-collaborative-filtering/

from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Although collaborative Filtering is very commonly used in recommenders, some of the challenges that are faced while using it are the following:

Collaborative filtering can lead to some problems like cold start for new items that are added to the list. Until someone rates them, they don’t get recommended.

Data sparsity can affect the quality of user-based recommenders and also add to the cold start problem mentioned above.

Scaling can be a challenge for growing datasets as the complexity can become too large. Item-based recommenders are faster than user-based when the dataset is large.

With a straightforward implementation, you might observe that the recommendations tend to be already popular, and the items from the long tail section might get ignored.